In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

import scipy
from scipy import spatial
from scipy.spatial import cKDTree

import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn import clone
from sklearn.externals.six.moves import xrange

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

## Overstock-SupplyChain-Understock (OSU) Correction

In [ ]:
# %%time

# dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
# dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

# dat = dat0.copy()
# dat = dat[['consumer_order_date', 'article_number', 'gross_demand_quantity', 'Clearance', 'season']]

# # Subset to non-clearance transactions only --- change here for David/Mike's request
# dat = dat[dat['Clearance'] == 0]

# #dat = dat[dat['article_number'] == 'M20325']

# # 'aggregate' to weekly sums by article for buy_availability merge and adjustment
# dat.set_index('consumer_order_date', inplace = True)

# dat = dat[['article_number', 'season', 'gross_demand_quantity']].groupby(['article_number', 'season']).resample('W').sum()
# dat.reset_index(inplace=True)

# # Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
# dat['week'] = [t.week for t in dat['consumer_order_date']]
# dat['year'] = [t.year for t in dat['consumer_order_date']]

In [ ]:
# dat.to_csv('dat_all_transactions_weekly.csv')

dat = pd.read_csv('dat_all_transactions_weekly.csv', low_memory=False, index_col = 0)
dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

In [ ]:
# Stock data
dat_stock16 = pd.read_csv('Stock Data/Stock data 2016.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock17 = pd.read_csv('Stock Data/Stock data 2017.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock18 = pd.read_csv('Stock Data/Stock data 2018.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock19 = pd.read_csv('Stock Data/Stock data 2019.csv', low_memory=False, index_col = 0) # *** DATA ***

dat_stock = pd.concat([dat_stock16, dat_stock17, dat_stock18, dat_stock19])
del dat_stock16, dat_stock17, dat_stock18, dat_stock19
dat_stock['min_date_of_week'] = pd.to_datetime(dat_stock['min_date_of_week'])

# dat_stock = dat_stock[dat_stock['article_number'] == 'M20325']

In [ ]:
# Tidy
dat_stock.reset_index(inplace = True)
dat_stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
dat_stock.rename(columns = {'min_date_of_week': 'date', 'avg(buy_availability)': 'buy_availability'}, inplace = True)

# Filter to SS20_range carryovers
# dat_stock = dat_stock[[(a in SS20_range) for a in dat_stock['article_number']]]

dat_stock['date'] = pd.to_datetime(dat_stock['date'])

# For merging with transaction data
dat_stock['week'] = [t.week for t in dat_stock['date']]
dat_stock['year'] = [t.year for t in dat_stock['date']]

In [ ]:
min_date = dat_stock['date'].min()
dat = dat[dat['consumer_order_date'] > min_date]

In [ ]:
# Merge weekly demand df and buy_availability df -- for understock correction
dat = pd.merge(dat, dat_stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'],
               how = 'left')

In [ ]:
dat.to_csv('NaNs_for_Joerian.csv')

In [ ]:
dat['buy_availability'] = dat['buy_availability'].fillna(1)

In [ ]:
# ----- Calculate *observed* full season gross_demand_quantity per article -----

dat_season = pd.DataFrame(dat.groupby(['article_number', 'season'])['gross_demand_quantity'].sum())
dat_season.rename(columns = {'gross_demand_quantity':'season_gross_demand_quantity'}, inplace = True)

In [ ]:
# WEEKLY averages for articles when buy_availability > 0.35
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number', 'season'])['gross_demand_quantity'].
    mean())

In [ ]:
dat_stocked.rename(columns = {'gross_demand_quantity':'corrected_weekly_avg_gross_demand_quantity'}, inplace= True)

In [ ]:
# Extend to full season (26 weeks) to estimate full season demand
dat_stocked['corrected_gross_demand_quantity'] = 26*dat_stocked['corrected_weekly_avg_gross_demand_quantity'] 

In [ ]:
dat_season = pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer')
dat_season.reset_index(inplace = True)

In [ ]:
dat_season = dat_season[(dat_season['season'] != 'FW16') & 
                        (dat_season['season'] != 'SS16') &
                        (dat_season['season_gross_demand_quantity'] > 50)
                       ]

In [ ]:
# PUT JOERIAN ON THIS

dat_season[dat_season['corrected_gross_demand_quantity'].isna()] 
dat[dat['article_number'] == 'AC7085'] # e.g.

# How many NaN
dat[dat['buy_availability'].isna()]

In [ ]:
dat = pd.merge(dat, dat_season[['article_number', 'season', 'corrected_weekly_avg_gross_demand_quantity']])

dat['OSU_gross_demand_quantity'] = np.where(dat['buy_availability'] < 0.35,
                                            dat['corrected_weekly_avg_gross_demand_quantity'],
                                            dat['gross_demand_quantity']
                                           )

dat = dat[['article_number', 'season', 'consumer_order_date', 
           'gross_demand_quantity', 'buy_availability', 'OSU_gross_demand_quantity']].round()


In [ ]:
len(dat.article_number.unique())
dat.season.value_counts()
len(dat.consumer_order_date.unique())

In [ ]:
dat.to_csv('weekly_OSU_transaction_data.csv')

In [ ]:
# dat = dat[dat['article_number'] == 'M20325']
# dat_stocked = dat_stocked[dat_stocked['article_number'] == 'M20325']